In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc

2023-12-14 00:25:48.295621: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-14 00:25:51.344575: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
f=h5py.File('/eos/home-w/wsherman/AD_Work/n_tuples/40MHZ_data/background_for_training.h5','r')
Dataset=np.array(f["Particles"])

truthtable=[]

threshold=50
for i, batch in enumerate(Dataset):
  if np.sum(batch[:,0])>=threshold:
    truthtable+=[1]
  else:
    truthtable+=[0]

event_pt_br=[]
Data_Test_full=Dataset[2000001:3600000,:,:]
for j, br_1 in enumerate(Data_Test_full):
  event_pt_br+=[np.sum(br_1[:,0])]

for i, batch in enumerate(Dataset):
  pt_sum=0
  for j, particle in enumerate(Dataset[i,:,:]):
    if particle[3]!=0:
      pt_sum+=particle[0]
  for j, particle in enumerate(Dataset[i,:,:]):
    particle[0]=particle[0]/pt_sum

Data_Train=Dataset[0:2000000,:,0:3]
Data_Test=Dataset[2000001:3600000,:,0:3]
Test_Truth=truthtable[2000001:3600000]
Data_Validate=Dataset[3600001:4000000,:,0:3]

In [3]:
def create_AE(input_dim, h_dim_1, h_dim_2, latent_dim):
    # Encoder
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(h_dim_1, activation='relu')(inputs)
    x = layers.Dense(h_dim_2, activation='relu')(x)
    z = layers.Dense(latent_dim, activation='relu')(x)

    # Decoder
    x = layers.Dense(h_dim_2, activation='relu')(z)
    x = layers.Dense(h_dim_1, activation='relu')(z)
    outputs = layers.Dense(input_dim)(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [4]:
def loss_fn(y_true, y_pred):
    """masked mse"""
    mask = K.cast(K.not_equal(y_true, 0), K.floatx())
    squared_difference = K.square(mask * (y_pred - y_true))
    return K.mean(squared_difference)

In [10]:
Data_Train_Flat=np.reshape(Data_Train,(-1,57))
Data_Val_Flat=np.reshape(Data_Validate,(-1,57))

In [11]:

INPUT_DIM = Data_Train_Flat.shape[1]
H_DIM_1 = 32
H_DIM_2 = 16
LATENT_DIM = 8
DNN_AE = create_AE(input_dim=INPUT_DIM, h_dim_1=H_DIM_1, h_dim_2=H_DIM_2, latent_dim=LATENT_DIM)
DNN_AE.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 57)]              0         
                                                                 
 dense_18 (Dense)            (None, 32)                1856      
                                                                 
 dense_19 (Dense)            (None, 16)                528       
                                                                 
 dense_20 (Dense)            (None, 8)                 136       
                                                                 
 dense_22 (Dense)            (None, 32)                288       
                                                                 
 dense_23 (Dense)            (None, 57)                1881      
                                                                 
Total params: 4,689
Trainable params: 4,689
Non-trainable p

In [13]:
DNN_AE.compile(optimizer='adam', loss=loss_fn)

STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 50
BATCH_SIZE = 1024

history = DNN_AE.fit(x=Data_Train_Flat, y=Data_Train_Flat, validation_data=(Data_Val_Flat,Data_Val_Flat), epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks)

Epoch 1/50
1954/1954 [==============================] - 11s 4ms/step - loss: 0.0193 - val_loss: 0.0056 - lr: 0.0010
Epoch 2/50
1954/1954 [==============================] - 8s 4ms/step - loss: 0.0051 - val_loss: 0.0048 - lr: 0.0010
Epoch 3/50
1954/1954 [==============================] - 8s 4ms/step - loss: 0.0045 - val_loss: 0.0045 - lr: 0.0010
Epoch 4/50
1954/1954 [==============================] - 8s 4ms/step - loss: 0.0043 - val_loss: 0.0042 - lr: 0.0010
Epoch 5/50
1954/1954 [==============================] - 8s 4ms/step - loss: 0.0041 - val_loss: 0.0039 - lr: 0.0010
Epoch 6/50
1954/1954 [==============================] - 8s 4ms/step - loss: 0.0039 - val_loss: 0.0038 - lr: 0.0010
Epoch 7/50
1954/1954 [==============================] - 8s 4ms/step - loss: 0.0038 - val_loss: 0.0038 - lr: 0.0010
Epoch 8/50
1954/1954 [==============================] - 8s 4ms/step - loss: 0.0038 - val_loss: 0.0037 - lr: 0.0010
Epoch 9/50
1954/1954 [==============================] - 8s 4ms/step - loss: 0.0

In [14]:
DNN_AE.save('/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/40MHZ_norm_DNN.keras')